<a href="https://colab.research.google.com/github/Ayanlola2002/Data-Science-Nigeria/blob/master/DSN_BOOTCAMP_2020_catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [674]:
#mounting Gdrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [675]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid
from itertools import product, chain
#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer
#imp= IterativeImputer()
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV,StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMModel,LGBMClassifier
from sklearn.ensemble import ExtraTreesClassifier
!pip install catboost
from catboost import Pool, CatBoostClassifier, cv
!pip install vecstack
#!pip install catboost
from vecstack import stacking
#pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

In [676]:
train_df=pd.read_csv('/content/drive/My Drive/DSN_BOOTCAMP/Train.csv')
test_df=pd.read_csv('/content/drive/My Drive/DSN_BOOTCAMP/Test.csv')

In [677]:

train_df.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,NaN,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.00,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,NaN,NaN,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.19,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [678]:
test_df.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,0.0,1649749.0,5446.0,0.0,48187375,114686147.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1086.0,6204.0,2676.0,60.0,14691.0,9794.0,NaN,238.0,1.8334,0.0,NaN,22.0,2.0,14.0,0.0,NaN,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,18308285.0,622789.0,1025793.0,1396510.0,34694910,87479487.0,1.0,1.0,1.0,2.0,2.0,142.560,NaN,NaN,570.585,3082.5,10221.0,6814.0,60.0,11955.0,3528.0,NaN,1216.0,19.5000,3.0,NaN,30.0,18.0,14.0,2.0,NaN,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,0.0,118256.0,NaN,NaN,21973443,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,852.0,4836.0,NaN,NaN,NaN,7.8334,0.0,NaN,2.0,NaN,0.0,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,493641.0,1391064.0,1176725.0,1220293.0,16576457,NaN,2.0,2.0,2.0,3.0,3.0,109.005,13050.00,NaN,109.005,1414.5,4197.0,2798.0,60.0,14964.0,NaN,NaN,248.0,9.0000,1.0,1.0,36.0,0.0,26.0,0.0,NaN,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,5752921.0,271133.0,613064.0,4779643.0,48187375,72049802.0,0.0,0.0,0.0,3.0,3.0,139.065,469.29,469.29,15.180,2724.0,9855.0,6570.0,182.0,11679.0,5900.0,NaN,2388.0,16.6666,1.0,0.0,24.0,28.0,20.0,0.0,NaN,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


In [679]:
Application_id=test_df['Applicant_ID']

In [680]:
 #train_df.dropna(thresh=train_df.shape[1]*0.7,how='all',axis=0,inplace=True)

In [681]:

print(train_df.shape)
print(test_df.shape)

(56000, 52)
(24000, 51)


In [682]:
train_df.shape

(56000, 52)

In [683]:
train_df.default_status.unique()

array(['no', 'yes'], dtype=object)

In [684]:
#target varriable
train_df.loc[train_df.default_status=='no','default_status']=0
#target varriable
train_df.loc[train_df.default_status=='yes','default_status']=1

In [685]:
#saving the order Id column and predictor column
#Application_id=test_df['Applicant_ID']
y_train=train_df.iloc[:,-1].astype('int')

In [686]:
y_train

0        0
1        0
2        1
3        0
4        0
        ..
55995    0
55996    1
55997    0
55998    0
55999    0
Name: default_status, Length: 56000, dtype: int64

In [687]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 52 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Applicant_ID    56000 non-null  object 
 1   form_field1     53471 non-null  float64
 2   form_field2     52156 non-null  float64
 3   form_field3     55645 non-null  float64
 4   form_field4     55645 non-null  float64
 5   form_field5     55645 non-null  float64
 6   form_field6     42640 non-null  float64
 7   form_field7     50837 non-null  float64
 8   form_field8     42640 non-null  float64
 9   form_field9     47992 non-null  float64
 10  form_field10    55645 non-null  float64
 11  form_field11    24579 non-null  float64
 12  form_field12    46105 non-null  float64
 13  form_field13    50111 non-null  float64
 14  form_field14    56000 non-null  int64  
 15  form_field15    33525 non-null  float64
 16  form_field16    42964 non-null  float64
 17  form_field17    44849 non-null 

In [688]:
train_df.form_field47.unique()

array(['charge', 'lending'], dtype=object)

In [689]:
(train_df.form_field6.isnull().sum()/len(train_df))*100

23.857142857142858

In [690]:
#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train_df.drop("Applicant_ID", axis = 1, inplace = True)
test_df.drop("Applicant_ID", axis = 1, inplace = True)

In [691]:
# let's delete the last column from the dataset to  concat train and test
train_df = train_df.drop(['default_status'], axis = 1)

In [692]:
train_df.dtypes

form_field1     float64
form_field2     float64
form_field3     float64
form_field4     float64
form_field5     float64
form_field6     float64
form_field7     float64
form_field8     float64
form_field9     float64
form_field10    float64
form_field11    float64
form_field12    float64
form_field13    float64
form_field14      int64
form_field15    float64
form_field16    float64
form_field17    float64
form_field18    float64
form_field19    float64
form_field20    float64
form_field21    float64
form_field22    float64
form_field23    float64
form_field24    float64
form_field25    float64
form_field26    float64
form_field27    float64
form_field28    float64
form_field29    float64
form_field30    float64
form_field31    float64
form_field32    float64
form_field33    float64
form_field34    float64
form_field35    float64
form_field36    float64
form_field37    float64
form_field38    float64
form_field39    float64
form_field40    float64
form_field41    float64
form_field42    

In [693]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 50 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   form_field1   22890 non-null  float64
 1   form_field2   22291 non-null  float64
 2   form_field3   23854 non-null  float64
 3   form_field4   23854 non-null  float64
 4   form_field5   23854 non-null  float64
 5   form_field6   18396 non-null  float64
 6   form_field7   21769 non-null  float64
 7   form_field8   18396 non-null  float64
 8   form_field9   20600 non-null  float64
 9   form_field10  23853 non-null  float64
 10  form_field11  10602 non-null  float64
 11  form_field12  19817 non-null  float64
 12  form_field13  21537 non-null  float64
 13  form_field14  24000 non-null  int64  
 14  form_field15  14408 non-null  float64
 15  form_field16  18526 non-null  float64
 16  form_field17  19305 non-null  float64
 17  form_field18  19631 non-null  float64
 18  form_field19  24000 non-nu

In [694]:
train_df

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,3436.0,0.28505,1.6560,0.0000,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335
1,3456.0,0.67400,0.2342,0.0000,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436
2,3276.0,0.53845,3.1510,0.0000,6.282,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640
3,3372.0,0.17005,0.5050,0.0000,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,NaN,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.00,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,NaN,NaN,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729
4,3370.0,0.77270,1.1010,0.0000,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.19,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55995,3740.0,0.01730,0.0000,0.0000,0.000,770998.0,9637475.0,4047934.0,11641992.0,19910965.0,NaN,983003.0,25985589.0,48187375,86615907.0,0.0,0.0,0.0,0.0,0.0,31.740,1662.15,0.00,15.810,2815.5,7299.0,4866.0,60772.0,15969.0,1034.0,NaN,864.0,21.6666,0.0,0.0,4.0,8.0,8.0,0.0,NaN,NaN,0.000000,9.09,0.730040,NaN,0.0,lending,591.048913,0.000000,0.241598
55996,3360.0,2.01145,0.6252,0.0000,0.000,NaN,927765.0,NaN,NaN,1849306.0,571052.0,NaN,0.0,16576457,NaN,NaN,NaN,0.0,0.0,1.0,NaN,13350.00,NaN,NaN,1840.5,5475.0,3650.0,60.0,11589.0,1642.0,11589.0,242.0,14.6666,0.0,NaN,28.0,0.0,34.0,0.0,158.0250,136.5135,0.297913,30.30,0.498208,NaN,0.0,lending,NaN,0.000000,NaN
55997,3500.0,0.76640,0.0000,0.0000,0.000,118645.0,3662435.0,3662435.0,3585024.0,704090.0,NaN,2765401.0,4226874.0,21973443,87508273.0,0.0,0.0,0.0,2.0,2.0,119.565,2206.95,12690.00,119.565,1059.0,3924.0,2616.0,2980.0,12135.0,NaN,NaN,232.0,22.3334,0.0,0.0,6.0,0.0,4.0,0.0,NaN,NaN,0.550000,3.03,0.450512,0.0,0.0,lending,1252.474467,2.000000,0.163153
55998,3280.0,0.05235,2.0916,2.2212,0.000,NaN,3458599.0,NaN,115533.0,3458599.0,508812.0,NaN,0.0,38549900,20238892.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,2373.0,NaN,NaN,60.0,25641.0,852.0,NaN,104.0,57.5000,1.0,NaN,10.0,2.0,8.0,1.0,NaN,150.0000,0.942854,0.00,0.705048,1.0,0.0,charge,NaN,0.000000,0.002997


In [695]:
#choose the features we want to train, just forget the float data
#cate_features_index = np.where(train_df.dtypes != float)[0]


In [696]:
cate_features_index=np.array([46])
#cate_features_index=[46]

In [697]:
train_df.shape


(56000, 50)

In [698]:
# lets concat the train and test sets for preprocessing and visualizations
all_data = pd.concat([train_df, test_df], axis = 0)

# let's check the shape
all_data.shape

(80000, 50)

In [699]:
'''def impute_with_median (df):
    """Iterate through columns of Pandas DataFrame.
    Where NaNs exist replace with median"""
    
    # Get list of DataFrame column names
    cols = list(df)
    # Loop through columns
    for column in cols:
        # Transfer column to independent series
        col_data = df[column]
        # Look to see if there is any missing numerical data
        missing_data = sum(col_data.isna())
        if missing_data > 0:
            # Get median and replace missing numerical data with median
            col_median = col_data.median()
            col_data.fillna(col_median, inplace=True)
            df[column] = col_data
    return df  '''

'def impute_with_median (df):\n    """Iterate through columns of Pandas DataFrame.\n    Where NaNs exist replace with median"""\n    \n    # Get list of DataFrame column names\n    cols = list(df)\n    # Loop through columns\n    for column in cols:\n        # Transfer column to independent series\n        col_data = df[column]\n        # Look to see if there is any missing numerical data\n        missing_data = sum(col_data.isna())\n        if missing_data > 0:\n            # Get median and replace missing numerical data with median\n            col_median = col_data.median()\n            col_data.fillna(col_median, inplace=True)\n            df[column] = col_data\n    return df  '

In [700]:
data_joined=all_data.replace(np.nan,-1)

In [701]:
data_joined.shape

(80000, 50)

In [702]:

data_joined.isnull().sum()

form_field1     0
form_field2     0
form_field3     0
form_field4     0
form_field5     0
form_field6     0
form_field7     0
form_field8     0
form_field9     0
form_field10    0
form_field11    0
form_field12    0
form_field13    0
form_field14    0
form_field15    0
form_field16    0
form_field17    0
form_field18    0
form_field19    0
form_field20    0
form_field21    0
form_field22    0
form_field23    0
form_field24    0
form_field25    0
form_field26    0
form_field27    0
form_field28    0
form_field29    0
form_field30    0
form_field31    0
form_field32    0
form_field33    0
form_field34    0
form_field35    0
form_field36    0
form_field37    0
form_field38    0
form_field39    0
form_field40    0
form_field41    0
form_field42    0
form_field43    0
form_field44    0
form_field45    0
form_field46    0
form_field47    0
form_field48    0
form_field49    0
form_field50    0
dtype: int64

In [703]:
#label encoding categorical varriables

le = LabelEncoder()
data_joined['form_field47'] = le.fit_transform(data_joined['form_field47'])
# removing unnecassary columns

data_joined =data_joined.drop(['form_field47'], axis = 1)

In [704]:

# separating train and test datasets from data

x_train = data_joined.iloc[:56000,:]
x_test = data_joined.iloc[56000:,:]

# checking the shape of train and test
print("Shape of train :", x_train.shape)
print("Shape of test :", x_test.shape)

Shape of train : (56000, 49)
Shape of test : (24000, 49)


In [705]:
# train and valid sets from train
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.2, random_state = 0)

# checking the shapes
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(44800, 49)
(44800,)
(11200, 49)
(11200,)


In [706]:

# standardization techniques

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_valid = sc.transform(x_valid)
x_test = sc.transform(x_test)

In [707]:
x_train

array([[ 0.1891987 ,  0.99784869, -0.48442958, ..., -0.14446114,
        -0.02161081, -0.09206707],
       [ 0.18652178, -0.50140915, -0.48442958, ..., -0.14469411,
        -0.57284651, -0.09206718],
       [ 0.03929116,  0.57396792, -0.48442958, ..., -0.14434129,
         2.183332  , -0.09206715],
       ...,
       [ 0.3739062 , -1.63523494, -0.48442958, ..., -0.14469411,
        -0.02161081, -0.09206738],
       [ 0.56932139, -0.46898174, -0.48442958, ...,  0.53537883,
        -0.57284651, -0.09206715],
       [ 0.02055271,  3.15733329,  0.05766856, ..., -0.14467778,
         1.36336634, -0.09206719]])

In [708]:
import hyperopt
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval

In [709]:
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score, classification_report
def cross_val(X, y, X_test, param, n_splits=10):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
    
    acc = []
    predict = None
    
    for tr_ind, val_ind in skf.split(X, y):
        X_train = X[tr_ind]
        y_train = y[tr_ind]
        
        X_valid = X[val_ind]
        y_valid = y[val_ind]
        
        clf = CatBoostClassifier(iterations=1000,
                                loss_function = param['loss_function'],
                                depth=param['depth'],
                                l2_leaf_reg = param['l2_leaf_reg'],
                                eval_metric = 'AUC',
                                leaf_estimation_iterations = 10,
                                use_best_model=True,
                                logging_level='Silent',
                                task_type = 'GPU'
        )
        
        clf.fit(X_train, 
                y_train,
                eval_set=(X_valid, y_valid)
        )
        c_probs = clf.predict_proba(X_valid)
#         y_pred = clf.predict(X_valid)
        auc = roc_auc_score(y_valid, c_probs[:, 1])
#         accuracy = accuracy_score(y_valid, y_pred)
        acc.append(auc)
    return sum(acc)/n_splits

In [710]:

def catboost_GridSearchCV(X, y, X_test, params, n_splits=10):
    ps = {'acc':0,
          'param': []
    }
    
    predict=None
    
    for prms in tqdm(list(ParameterGrid(params)), ascii=True, desc='Params Tuning:'):
                          
        acc = cross_val(X, y, X_test, prms, n_splits=10)

        if acc>ps['acc']:
            ps['acc'] = acc
            ps['param'] = prms
    print('Acc: '+str(ps['acc']))
    print('Params: '+str(ps['param']))
    
    return ps['param']

In [715]:

params = {'depth':np.arange(5, 16, 1, dtype=int),
              'loss_function': ['Logloss'],
              'l2_leaf_reg':np.logspace(-20, -19, 3),
              'learning_rate':np.arange(0.05, 0.31, 0.05),
              'colsample_bylevel':np.arange(0.3, 0.8, 0.1),
              'n_estimators':     np.arange(100,2600,100),
              'eval_metric':      ['AUC']
    }





In [716]:
cat_features=[46]
param = catboost_GridSearchCV(x_train, y_train, x_test, params)

Params Tuning::   0%|          | 0/24750 [00:00<?, ?it/s]


KeyError: ignored

In [584]:
#let us make the catboost model, use_best_model params will make the model prevent overfitting
model = CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42)

CatBoostClassifier(iterations=2500,
                            loss_function = param['loss_function'],
                            depth=param['depth'],
                            l2_leaf_reg = param['l2_leaf_reg'],
                            eval_metric = 'AUC',
                            leaf_estimation_iterations = 10,
                            task_type = 'GPU'

In [585]:
#now just to make the model to fit the data
model.fit(x_train,y_train,cat_features=cate_features_index,eval_set=(x_valid,y_valid))

Learning rate set to 0.081054
0:	learn: 0.7783259	test: 0.7751786	best: 0.7751786 (0)	total: 41.4ms	remaining: 41.3s
1:	learn: 0.7887723	test: 0.7873214	best: 0.7873214 (1)	total: 79.2ms	remaining: 39.5s
2:	learn: 0.7901786	test: 0.7869643	best: 0.7873214 (1)	total: 115ms	remaining: 38.4s
3:	learn: 0.7930357	test: 0.7895536	best: 0.7895536 (3)	total: 152ms	remaining: 37.8s
4:	learn: 0.7940179	test: 0.7903571	best: 0.7903571 (4)	total: 188ms	remaining: 37.5s
5:	learn: 0.7938170	test: 0.7900893	best: 0.7903571 (4)	total: 239ms	remaining: 39.6s
6:	learn: 0.7938393	test: 0.7889286	best: 0.7903571 (4)	total: 275ms	remaining: 39s
7:	learn: 0.7945313	test: 0.7917857	best: 0.7917857 (7)	total: 309ms	remaining: 38.4s
8:	learn: 0.7953125	test: 0.7917857	best: 0.7917857 (7)	total: 345ms	remaining: 38s
9:	learn: 0.7965402	test: 0.7936607	best: 0.7936607 (9)	total: 381ms	remaining: 37.7s
10:	learn: 0.7974554	test: 0.7943750	best: 0.7943750 (10)	total: 416ms	remaining: 37.4s
11:	learn: 0.7982366	tes

In [511]:
#for the data is not so big, we need use the cross-validation(cv) for the model, to find how
#good the model is ,I just use the 10-fold cv
#cv_data = cv(Pool(x_train,y_train,cat_features=cate_features_index),model.get_params(),fold_count=10)

In [512]:
#show the acc for the model
#print('the best cv accuracy is :{}'.format(np.max(cv_data["b'Accuracy'_test_avg"])))

In [513]:
#show the model test acc, but you have to note that the acc is not the cv acc,
#so recommend to use the cv acc to evaluate your model!
#print('the test accuracy is :{:.6f}'.format(accuracy_score(y_valid,model.predict_proba(x_valid))))

In [586]:

y_pred = model.predict_proba(test_df)[:,1]

In [587]:

#Prepare our Submission file
my_submission = pd.DataFrame({'Application ID':Application_id, 'default_status':y_pred})
my_submission.to_csv('submission_ayanlola_simple_median_catboost2_only.csv', index=False)